# СЕМИНАР. Сверточные нейронные сети и Keras на примере CIFAR-10

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Подготовка данных</a></li>
        <li><a href="#2">Построение сверточной нейронной сети</a></li>
    </ol>
    <a href="#3">Источники</a>
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from keras.datasets import mnist,cifar10
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Activation, Flatten
from keras.optimizers import SGD, Adam
from keras.initializers import TruncatedNormal,Constant
from keras.preprocessing.image import ImageDataGenerator
from keras import utils
from keras.utils import np_utils

np.random.seed(555)

cifar10_path = os.getcwd() + "/data/cnn-cifar10/" + "cifar-10-python.tar.gz"
os.makedirs(os.getcwd() + "/data/cnn-cifar10", exist_ok=True)

<a name="1"></a>
## 1. Подготовка данных

Загрузка датасета

In [ ]:
#fn = cifar10.get_file(cifar10_path, "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz")
(x_train, y_train),(x_test, y_test) = cifar10.load_data() # датасет загрузится в ~/.keras/datasets/
classes = {0:"airplane", 1:"automobile", 2:"bird", 3:"cat", 4:"deer", 
           5:"dog", 6:"frog", 7:"horse", 8:"ship", 9:"truck"}

Предобработка данных

In [ ]:
# масштабируем значения пикселей в диапазон от 0.0 до 1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [ ]:
# вывод рандомного примера из обучающего множества
ind = np.random.choice(len(x_train))
plt.imshow(x_train[ind])
print("Класс: %s" % classes[np.argmax(y_train[ind])])

<a name="2"></a>
## 2. Построение сверточной нейронной сети

Параметры для обучения и тестирования нейронной сети

In [ ]:
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
classes = 10
# Количество эпох для обучения
num_epochs = 30
# Размер изображения
img_h, img_w = 32, 32
# Количество каналов в изображении
img_ch = 3

Строим модель сверточной нейронной сети

In [ ]:
# Создаем последовательную модель
cnn2 = Sequential()
# Первый сверточный слой
cnn2.add(Conv2D(64, (3, 3), padding='same',
                        input_shape=(32, 32, 3), activation='relu'))
# Второй сверточный слой
cnn2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# Первый слой подвыборки
cnn2.add(MaxPool2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
cnn2.add(Dropout(0.4))

# Третий сверточный слой
cnn2.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
# Четвертый сверточный слой
cnn2.add(Conv2D(256, (3, 3), activation='relu'))
# Второй слой подвыборки
cnn2.add(MaxPool2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
cnn2.add(Dropout(0.4))
# Слой преобразования данных из двумерного представления в вектор
cnn2.add(Flatten())
# Полносвязный слой для классификации
cnn2.add(Dense(1024, activation='relu'))
# Слой регуляризации Dropout
cnn2.add(Dropout(0.7))
# Выходной полносвязный слой Softmax
cnn2.add(Dense(classes, activation='softmax'))

Вывод информации о структуре полученной сети

In [ ]:
cnn2.summary()

Выбор оптимизатора и комапиляция модели

In [ ]:
sgd = Adam()
cnn2.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

Обучение: 90%-обучающая выборка, 10%-валидационная выборка

In [ ]:
# Обучение модели (без генератора изображений)
%%time
hist = cnn2.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.1,
          shuffle=True,
          verbose=1)

In [ ]:
# Визуализация процесса обучения - построение графиков зависимости ошибки и точности от эпохи
plt.plot(hist.history["loss"], label="Ошибка на обучающей выборке")
plt.plot(hist.history["val_loss"], label="Ошибка на валидационной выборке")
plt.xlabel("Эпоха")
plt.ylabel("Ошибка")
plt.legend()
plt.show()
plt.plot(hist.history["acc"], label="Точность на обучающей выборке")
plt.plot(hist.history["val_acc"], label="Точность на валидационной выборке")
plt.xlabel("Эпоха")
plt.ylabel("Точность")
plt.legend()
plt.show()

Модель можно обучать дальше для повышения точности классификации (до тех пор пока не пойдет переобучение сети)

Оценка точности классификации

In [ ]:
# Тестирование модели
scores = cnn2.evaluate(x_test, y_test, verbose=2)
print("Точность: %.2f%%" % (scores[1]*100))

Сохранение сети в файл и загрузка из файла

In [ ]:
model_path = "./models/cnn-mnist-cifar10/cnn_cifar10_model.h5"
# сохранение обученной модели в файл
cnn2.save(model_path)

# загрузка сохраненной модели из файла
#cnn2 = load_model(model_path)
#scores = cnn2.evaluate(x_test, y_test, verbose=0)
#print("Точность: %.2f%%" % (scores[1]*100))

In [ ]:
# вывод предсказания для произвольного примера и его истинную метку класса
t = np.random.choice(len(x_test))
plt.imshow(x_test[t])
print("Истинный класс: " + classes[np.argmax(y_test[t])])
print("Предсказанный класс: " + classes[np.argmax(cnn2.predict(np.expand_dims(x_test[t], axis=0)))])

<a name="3"></a>
## Источники

<div>
<a href="https://www.tensorflow.org/tutorials/deep_cnn">Convolutional Neural Networks</a><br>
<a href="http://cs231n.github.io/convolutional-networks/">Convolutional Neural Networks (CNNs / ConvNets)</a><br>
<a href="http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf">ImageNet Classification with Deep Convolutional Neural Networks (PDF)</a><br>
<a href="https://keras.io/getting-started/functional-api-guide/">Keras API Introduction</a><br>
<a href="https://keras.io/getting-started/sequential-model-guide/">Keras Sequential Model Guide</a><br>
<a href="https://keras.io/datasets/">Keras Datasets</a>
</div>